<a href="https://colab.research.google.com/github/ganeshhep/FCC/blob/main/strange_frag_tag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastjet
!pip install uproot
!pip install torch
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.4/871.4 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 638.7/638.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.5/363.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import uproot
import awkward as ak
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_geometric.utils import add_self_loops
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
import itertools
from fastjet import PseudoJet, JetDefinition, ClusterSequence, antikt_algorithm, sorted_by_pt
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_file_1 = '/content/drive/MyDrive/ML_HEP/Data_files/fcc_ee_h_ss.root'
data_file_2 = '/content/drive/MyDrive/ML_HEP/Data_files/fcc_ee_h_qq.root'
tree_1 = uproot.open(data_file_1)['events']
tree_2 = uproot.open(data_file_2)['events']

Mounted at /content/drive


In [ ]:
pid = ak.concatenate([tree_1['Particle.PDG'].array(), tree_2['Particle.PDG'].array()], axis = 0)
p_status = ak.concatenate([tree_1['Particle.generatorStatus'].array(), tree_2['Particle.generatorStatus'].array()], axis = 0)
charge = ak.concatenate([tree_1['Particle.charge'].array(), tree_2['Particle.charge'].array()], axis = 0)
m = ak.concatenate([tree_1['Particle.mass'].array(), tree_2['Particle.mass'].array()], axis = 0)
px = ak.concatenate([tree_1['Particle.momentum.x'].array(), tree_2['Particle.momentum.x'].array()], axis = 0)
py = ak.concatenate([tree_1['Particle.momentum.y'].array(), tree_2['Particle.momentum.y'].array()], axis = 0)
pz = ak.concatenate([tree_1['Particle.momentum.z'].array(), tree_2['Particle.momentum.z'].array()], axis = 0)
p_begin = ak.concatenate([tree_1['Particle.parents_begin'].array(), tree_2['Particle.parents_begin'].array()], axis = 0)
p_end = ak.concatenate([tree_1['Particle.parents_end'].array(), tree_2['Particle.parents_end'].array()], axis = 0)
p_ind = ak.concatenate([tree_1['Particle#0.index'].array(), tree_2['Particle#0.index'].array()], axis = 0)

In [ ]:
# Function to calculate energy
def energy(m, px, py, pz):
    E = np.sqrt( m**2 + px**2 + py**2 + pz**2)
    return E

In [ ]:
e = energy(m, px, py, pz) # Energy of particles

In [ ]:
def get_parent_ids(i, particle_ids, parent_indices, parents_begin, parents_end) :
    """
    Find parent ID and it's index in the particle IDs list.

    Parameters:
        i (integer): Index of the stable particle.
        particle_ids (list): A list of IDs of particles.
        parent_indices (list): A list of parent indices of particles.
        parents_begin (list): A list of indices of the first parent of each particle.
        parents_end (list): A list of indices of the last parent of each particle.

    Returns:
        parent_id (integer): Parent ID of the stable particle.
        parent_indx (integer): Index of the parent particle.

    """
    pb = parents_begin[i]
    pe = parents_end[i]

    if pb == pe :
      return 0
    else :
      for id in range(pb, pe) :
        parent_indx = parent_indices[id]
      return parent_indx

In [ ]:
def find_parent_and_daughter_ids(indices, particle_ids, all_particle_ids):
    """
    Find parent IDs and corresponding daughter IDs for each repeated index.

    Parameters:
        indices (list): A list of parent indices of stable particles.
        particle_ids (list): A list of IDs of stable particles.
        all_particle_ids (list): A list of all particle IDs.

    Returns:
        list: [[parent index, parent ID, list of daughter IDs],...,....].
    """
    # Count occurrences of each index
    counts = Counter(indices)

    # Identify repeated indices
    repeated_indices = {index for index, count in counts.items() if count >= 2}

    # Collect parent and daughter IDs for repeated indices
    parent_daughters = []
    for index in repeated_indices:
        parent_id = all_particle_ids[index]  # Get the parent ID using the index
        daughters = [particle_ids[i] for i in range(len(indices)) if indices[i] == index]
        parent_daughters.append([index, parent_id, daughters])

    return parent_daughters

In [ ]:
def get_graph_list(R, batch_size = 5000) : # R is the radius parameter
  graph_list = [] # List of graph datasets

  N = 200000 # number of events

  deta_mean = -0.0015069327248682111 # Mean of dijet eta distributions
  dpt_mean = 35.78125547420067 # Mean of dijet pt distributions
  dN_mean = 43.16249 # Mean of dijet N distributions

  deta_std = 1.1429906944033923 # Standard deviation of dijet eta distributions
  dpt_std = 17.15648916096553 # Standard deviation of dijet pt distributions
  dN_std = 12.49922825617246 # Standard deviation of dijet N distributions

  # Define Google Drive save path
  drive_path = "/content/drive/MyDrive/Graph_Datasets/"
  os.makedirs(drive_path, exist_ok=True)

  device = "cuda" if torch.cuda.is_available() else "cpu"

  for start in range(0, N, batch_size):
    end = min(start + batch_size, N)
    batch_graphs = []

    for n in range(start, end) :
      particles_list = [] # A list of all the particles in an event

      constituent_indx = [] # A list of indices of constitents of the dijet

      parent_indices = [] # A list of parent indices of constitents of the dijet

      pt_norm = [] # A list of normalized transverse momentum of the constituents of the dijet

      ident = [] # A list containing identity of the particles

      node_ft = [] # Node feature matrix

      edge_ft = [] # Edge attribute matrix

      photon_energy = 0 # Total photon energy
      muon_energy = 0 # Total muon energy
      electron_energy = 0 # Total electron energy
      Ks_energy = 0 # Total reconstructed Ks meson energy
      lambda_energy = 0 # Total reconstructed lambda baryon energy
      lambdabar_energy = 0 # Total reconstructed lambdabar baryon energy

      sts = p_status[n] # Generator status of particles
      p_x = px[n] # X-momentum of particles
      p_y = py[n] # Y-momentum of particles
      p_z = pz[n] # Z-momentum of particles
      E = e[n] # Energy of particles
      pcharge = charge[n] # Charge of particles
      ids = pid[n] # PDG id of particles
      pinds = p_ind[n] # Parent indices of particles
      pbegin = p_begin[n] # Parent begin indices of particles
      pend = p_end[n] # Parent end indices of particles

      for i in range(len(p_x)) :
        part_p4 = PseudoJet(float(p_x[i]), float(p_y[i]), float(p_z[i]), float(E[i]))
        part_p4.set_user_index(i) # setting the index of the particle
        particles_list.append(part_p4)

      stable_p4 = [] # A list of stable particles in the event

      for i in range(len(particles_list)) :
        if sts[i] == 1 :
          stable_p4.append(particles_list[i])

      # Applying clustering of particles with anti-kt algorithm
      cluster = ClusterSequence(stable_p4, JetDefinition(antikt_algorithm, R))
      jet_set = sorted_by_pt(cluster.inclusive_jets())

      if len(jet_set) > 1 :
        # Dijet kinematics
        dijet = jet_set[0] + jet_set[1]
        d_m = dijet.m() # invariant mass
        d_pt = dijet.pt() # transverse momentum
        d_eta = dijet.eta() # pseudo rapidity
        d_e = dijet.e() # energy

        Dijet_constituents = jet_set[0].constituents() + jet_set[1].constituents() # Dijet constituents

        N_constituents = len(Dijet_constituents) # Total number of constituents N in the dijet

        for i in range(N_constituents) :
          constituent_indx.append(Dijet_constituents[i].user_index())

        constituent_ids = ids[constituent_indx]

        for j in constituent_indx :
          const_id = ids[j]

          if const_id == 22 :
            photon_energy += particles_list[j].e()
          if const_id == 13 :
            muon_energy += particles_list[j].e()
          if const_id == 11 :
            electron_energy += particles_list[j].e()

          parent_indx = get_parent_ids(j, ids, pinds, pbegin, pend)
          parent_indices.append(parent_indx)

          parent_id = ids[parent_indx]

          pt_norm.append(particles_list[j].pt()/d_pt) # Normalized pT

          p_charge = pcharge[j] # Particle charge

          if p_charge > 0 or parent_id == 3122 :
            ident.append(1)
          elif p_charge < 0 or parent_id == 310 :
            ident.append(-1)
          else :
            ident.append(0)

        pd = find_parent_and_daughter_ids(parent_indices, constituent_ids, ids)

        mes_count = 0 # Strange meson count
        bar_count = 0 # Strange baryon count

        for k in range(len(pd)) :
          pd_indx = pd[k][0]
          pd_id = pd[k][1]

          if pd_id == 310 :
            mes_count = 1
            Ks_energy += particles_list[pd_indx].e()
          if pd_id == 3122 :
            bar_count = 1
            lambda_energy += particles_list[pd_indx].e()
          if pd_id == -3122 :
            bar_count = 1
            lambdabar_energy += particles_list[pd_indx].e()

        for l in range(len(constituent_indx)) :
          node_ft.append([(d_eta - deta_mean)/deta_std, (d_pt - dpt_mean)/dpt_std, (N_constituents - dN_mean)/dN_std, photon_energy/d_e, muon_energy/d_e, electron_energy/d_e, Ks_energy/d_e, lambda_energy/d_e, lambdabar_energy/d_e, pt_norm[l], ident[l]])

        x = torch.tensor(node_ft, dtype = torch.float).to(device) # Node features matrix

        y = torch.tensor([mes_count, bar_count]).to(device) # Target features

        edge_index = torch.tensor(list(itertools.permutations(range(N_constituents), 2))).T.to(device) # Edge index

        src = edge_index[0].tolist()
        dst = edge_index[1].tolist()

        for i in range(len(src)) :
          node_i = src[i]
          node_j = dst[i]

          edge_ft.append([(d_eta - deta_mean)/deta_std, (d_pt - dpt_mean)/dpt_std, (N_constituents - dN_mean)/dN_std, photon_energy/d_e, muon_energy/d_e, electron_energy/d_e, Ks_energy/d_e, lambda_energy/d_e, lambdabar_energy/d_e, pt_norm[node_i], ident[node_i], pt_norm[node_j], ident[node_j]])

        edge_attr = torch.tensor(edge_ft, dtype = torch.float).to(device) # Edge attributes matrix

        graph = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y) # Graph data

        batch_graphs.append(graph)

    # Save batch graphs to Google Drive
    save_path = os.path.join(drive_path, f"graph_batch_{start}_{end}.pt")
    torch.save(batch_graphs, save_path)
    print(f"Saved batch: {save_path}")

  return None

In [ ]:
graph_list = get_graph_list(R = 0.8)

Saved batch: /content/drive/MyDrive/Graph_Datasets/graph_batch_0_5000.pt
Saved batch: /content/drive/MyDrive/Graph_Datasets/graph_batch_5000_10000.pt
Saved batch: /content/drive/MyDrive/Graph_Datasets/graph_batch_10000_15000.pt
Saved batch: /content/drive/MyDrive/Graph_Datasets/graph_batch_15000_20000.pt
Saved batch: /content/drive/MyDrive/Graph_Datasets/graph_batch_20000_25000.pt
Saved batch: /content/drive/MyDrive/Graph_Datasets/graph_batch_25000_30000.pt
Saved batch: /content/drive/MyDrive/Graph_Datasets/graph_batch_30000_35000.pt
Saved batch: /content/drive/MyDrive/Graph_Datasets/graph_batch_35000_40000.pt
Saved batch: /content/drive/MyDrive/Graph_Datasets/graph_batch_40000_45000.pt
Saved batch: /content/drive/MyDrive/Graph_Datasets/graph_batch_45000_50000.pt
Saved batch: /content/drive/MyDrive/Graph_Datasets/graph_batch_50000_55000.pt
Saved batch: /content/drive/MyDrive/Graph_Datasets/graph_batch_55000_60000.pt
Saved batch: /content/drive/MyDrive/Graph_Datasets/graph_batch_60000_6

In [ ]:
def load_graph_batches(drive_path = "/content/drive/MyDrive/Graph_Datasets/"):
    graph_list = []

    # Get all batch files
    batch_files = sorted([f for f in os.listdir(drive_path) if f.endswith(".pt")])

    for file in batch_files:
        file_path = os.path.join(drive_path, file)
        batch_graphs = torch.load(file_path)  # Load the batch
        graph_list.extend(batch_graphs)  # Add to full list

    return graph_list

In [ ]:
graph_list = load_graph_batches()

<ipython-input-9-373a04f493d8>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  batch_graphs = torch.load(file_path)  # Load the batch


In [11]:
len(graph_list)

200000

In [12]:
from sklearn.model_selection import train_test_split

# Separate signal and background
signal_graphs = graph_list[:100000]  # First 100K are H → s s̄
background_graphs = graph_list[100000:]  # Next 100K are H → q q̄

# Train-validation-test split (70-15-15)
train_signal, temp_signal = train_test_split(signal_graphs, test_size = 0.30, random_state = 42)
train_background, temp_background = train_test_split(background_graphs, test_size = 0.30, random_state = 42)

val_signal, test_signal = train_test_split(temp_signal, test_size = 0.50, random_state = 42)
val_background, test_background = train_test_split(temp_background, test_size = 0.50, random_state = 42)

# Merge signal and background in each split
train_data = train_signal + train_background
val_data = val_signal + val_background
test_data = test_signal + test_background

# DataLoader for each split
train_loader = DataLoader(train_data, batch_size = 64, shuffle = True)
val_loader = DataLoader(val_data, batch_size = 64, shuffle = False)
test_loader = DataLoader(test_data, batch_size = 64, shuffle = False)

print(f"Training set: {len(train_data)} graphs (Balanced: {len(train_signal)} signal, {len(train_background)} background)")
print(f"Validation set: {len(val_data)} graphs (Balanced: {len(val_signal)} signal, {len(val_background)} background)")
print(f"Test set: {len(test_data)} graphs (Balanced: {len(test_signal)} signal, {len(test_background)} background)")

Training set: 140000 graphs (Balanced: 70000 signal, 70000 background)
Validation set: 30000 graphs (Balanced: 15000 signal, 15000 background)
Test set: 30000 graphs (Balanced: 15000 signal, 15000 background)


In [13]:
class MPNN(MessagePassing) :
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr = 'mean')
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.edge_lin = torch.nn.Linear(13, out_channels)
        self.out_layer = torch.nn.Linear(out_channels, out_channels)

    def forward(self, x, edge_index, edge_attr) :
        x = self.lin(x)
        return self.propagate(edge_index, x = x, edge_attr = edge_attr)

    def message(self, x_j, edge_attr) :
        return x_j + self.edge_lin(edge_attr)

    def update(self, aggr_out) :
        return self.out_layer(aggr_out)

In [14]:
class GraphMPNN(torch.nn.Module) :
    def __init__(self, in_channels, hidden_channels, out_channels = 2) :  # 2 output neurons for classification
        super().__init__()
        self.mpnn = MPNN(in_channels, hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, data) :
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        x = self.mpnn(x, edge_index, edge_attr)
        x = global_mean_pool(x, batch)
        return self.fc(x)  # No softmax, since BCEWithLogitsLoss expects raw logits

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define model
model = GraphMPNN(in_channels = 11, hidden_channels = 16, out_channels = 2).to(device)

# Optimizer and Binary Cross-Entropy Loss
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
criterion = torch.nn.BCEWithLogitsLoss()  # Multi-label classification loss

def evaluate(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # No gradients needed for evaluation
        for batch in data_loader:
            out = model(batch)
            probs = torch.sigmoid(out)  # Convert logits to probabilities
            preds = (probs > 0.5).int()  # Convert probabilities to binary labels (0 or 1)
            correct += (preds == batch.y.view(-1, 2)).sum().item()
            total += batch.y.numel()  # Count total number of labels (in the batch)
    return correct / total

# Training loop with validation
best_val_acc = 0
for epoch in range(50):
    model.train()  # Set the model to training mode
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out, batch.y.view(-1, 2).float())
        loss.backward()
        optimizer.step()

    # Evaluate on validation set after each epoch
    val_acc = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Validation Accuracy: {val_acc * 100:.2f}%")

    # Save the best model based on validation accuracy
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")

# Load the best model for final testing
model.load_state_dict(torch.load("best_model.pth", weights_only = True))

# Evaluate on the test set
test_acc = evaluate(model, test_loader)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

Epoch 1, Loss: 0.0057, Validation Accuracy: 99.71%
Epoch 2, Loss: 0.0064, Validation Accuracy: 99.26%
Epoch 3, Loss: 0.0020, Validation Accuracy: 99.88%
Epoch 4, Loss: 0.0032, Validation Accuracy: 99.88%
Epoch 5, Loss: 0.0006, Validation Accuracy: 99.82%
Epoch 6, Loss: 0.0009, Validation Accuracy: 99.95%
Epoch 7, Loss: 0.0001, Validation Accuracy: 99.90%
Epoch 8, Loss: 0.0006, Validation Accuracy: 99.98%
Epoch 9, Loss: 0.0002, Validation Accuracy: 99.87%
Epoch 10, Loss: 0.0002, Validation Accuracy: 99.97%
Epoch 11, Loss: 0.0001, Validation Accuracy: 99.98%
Epoch 12, Loss: 0.0005, Validation Accuracy: 99.93%
Epoch 13, Loss: 0.0000, Validation Accuracy: 99.99%
Epoch 14, Loss: 0.0003, Validation Accuracy: 99.98%
Epoch 15, Loss: 0.0001, Validation Accuracy: 99.99%
Epoch 16, Loss: 0.0000, Validation Accuracy: 99.94%
Epoch 17, Loss: 0.0003, Validation Accuracy: 99.93%
Epoch 18, Loss: 0.0002, Validation Accuracy: 100.00%
Epoch 19, Loss: 0.0000, Validation Accuracy: 99.95%
Epoch 20, Loss: 0.03